# Retrieve climate data through the meteo.cat API

In [18]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt 
import time
import csv
import geojson
import folium
from folium.plugins import HeatMap
import datetime as dt
import dask
import dask.dataframe as dd


In [2]:
# Check usage
key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'

url = 'https://api.meteo.cat/quotes/v1/consum-actual'

headers = {'X-API-KEY': 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'}
requests.get(url, headers=headers).json()

{'client': {'nom': 'Pau Torrente Badia UB'},
 'plans': [{'nom': 'XEMA_750 OD',
   'periode': 'Mensual',
   'maxConsultes': 750,
   'consultesRestants': 0,
   'consultesRealitzades': 750},
  {'nom': 'Predicció_100',
   'periode': 'Mensual',
   'maxConsultes': 100,
   'consultesRestants': 100,
   'consultesRealitzades': 0},
  {'nom': 'XDDE_250',
   'periode': 'Mensual',
   'maxConsultes': 250,
   'consultesRestants': 250,
   'consultesRealitzades': 0},
  {'nom': 'Quota',
   'periode': 'Mensual',
   'maxConsultes': 300,
   'consultesRestants': 292,
   'consultesRealitzades': 8},
  {'nom': 'Referència Bàsic',
   'periode': 'Mensual',
   'maxConsultes': 2000,
   'consultesRestants': 2000,
   'consultesRealitzades': 0}]}

In [ ]:
# Get request to check resource availability  --> Just remove # to run

# url = 'https://api.meteo.cat/quotes/v1/consum-actual'

# headers = {'X-API-KEY': 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'}
# requests.get(url, headers=headers).json()

In [ ]:
# Example of a get request retrieval  --> Just remove # to run

# key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'
# url = 'https://api.meteo.cat/xema/v1' + '/estacions/mesurades/X8/2020/03/02'

# headers = {'Accept': 'application/json', 'X-API-KEY': key}

# r = requests.get(url, headers=headers)

# r.json()

In [27]:
# Request example with processing to retrieve rain and temperature for each timestamp and store them as dictionaries on a list  --> Just remove # to run

# key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'
# url = 'https://api.meteo.cat/xema/v1' + '/estacions/mesurades/X8/2020/03/02'

# headers = {'Accept': 'application/json', 'X-API-KEY': key}

# r = requests.get(url, headers=headers)

# data = r.json()[0]


# # 35 is for precipitation in mm and 32 for temperature in ºC

# precipitation = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 35][0]
# temperature = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 32][0]


# date_variables = [{'timestamp':int(dt.datetime.strptime(d['data'], '%Y-%m-%dT%H:%MZ').timestamp()), 'mm_precip':d['valor']} for d in precipitation]

# for i in range(len(date_variables)):
#     date_variables[i]['temperature'] = temperature[i]['valor']

# date_variables

# Station localization on a map using the folium library

In [2]:

gener_info = pd.read_csv('data/2021_01_Gener_BicingNou_INFORMACIO.csv')

gener_info.head()



,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,nearby_distance,cross_street,last_updated,ttl
0,1,"GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397978,2.180107,16,"GRAN VIA CORTS CATALANES, 760",8013,46,1000,NaN,1609455629,25
1,2,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395488,2.177198,17,"C/ ROGER DE FLOR, 126",8013,27,1000,NaN,1609455629,25
2,3,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394156,2.181331,11,"C/ NÀPOLS, 82",8013,27,1000,NaN,1609455629,25
3,4,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393317,2.181248,8,"C/ RIBES, 13",8013,21,1000,NaN,1609455629,25
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391103,2.180176,7,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8018,39,1000,NaN,1609455629,25


In [4]:
gener_estacions = pd.read_csv('data/2021_01_Gener_BicingNou_ESTACIONS.csv')

gener_estacions.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,last_updated,ttl
0,1,43,43,0,3,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
1,2,17,15,2,10,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
2,3,3,3,0,22,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
3,4,15,14,1,6,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
4,5,8,8,0,31,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22


In [5]:
int(gener_estacions.head()['last_reported'][0])

1609455406

In [9]:
# something like this should be done if we want to account for spain's time dts changes
import pytz
from datetime import datetime

# create a timezone object for the Spanish time zone
spain_tz = pytz.timezone('Europe/Madrid')


og = 1583503600

# create a datetime object representing a timestamp in UTC
utc_time = datetime.utcfromtimestamp(og)

# convert the UTC datetime object to the Spanish time zone
spain_time = utc_time.replace(tzinfo=pytz.utc).astimezone(spain_tz)

# convert the Spanish datetime object to a Unix timestamp
spain_timestamp = int(spain_time.timestamp())

# print the original and corrected timestamps
print("Original timestamp:", og)
print("Corrected timestamp:", spain_timestamp)

Original timestamp: 1583503600
Corrected timestamp: 1583503600


# WEATHER AND STATION DATA MERGE


In [6]:
weather_data = pd.read_csv('weather_data/weather.csv')
weather_data= weather_data.rolling(window=2,  step=1).mean()
weather_data['timestamp'] = weather_data['timestamp'].astype(int)#-1800
weather_data

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [29]:
# weather_data['datetime_formatted'] = 
5//2

2

In [13]:
dataset = pd.merge(left = gener_estacions, right = weather_data, left_on = 'wt_last_reported', right_on = 'timestamp',   how = 'left')

target = dataset['num_bikes_available']
df = dataset['']

In [3]:
import dask.dataframe as dd

data  = dd.read_csv('data/data_bicing_joined_HX.csv', assume_missing=True, delimiter=';')


In [4]:
import numpy as np

data['status'] = data['status'].apply(lambda x: 1 if x == 'IN_SERVICE' else 0, meta = ('status', 'str'))



In [17]:
data.head()

,station_id,name,year,month,day,hour,last_updated_formated,last_update_formated_H-1,last_update_formated_H-2,last_update_formated_H-3,...,is_installed,is_renting,is_returning,% Docks Availlable,% Docks Available H-4,% Docks Available H-3,% Docks Available H-2,% Docks Available H-1,status,ttl
0,290.0,"PL. DELS JARDINS D'ALFÀBIA, 1",2019.0,7.0,22.0,8.0,2019-07-22 08:59:55,2019-07-22 07:59:55,2019-07-22 06:59:55,2019-07-22 05:59:55,...,1.0,1.0,1.0,0.751131,0.352941,0.352941,0.352941,0.504902,1,30.0
1,271.0,"VIA BARCINO, 69",2022.0,6.0,10.0,21.0,2022-06-10 21:59:45,2022-06-10 20:59:45,2022-06-10 19:59:45,2022-06-10 18:59:45,...,1.0,1.0,1.0,0.769231,0.753968,0.659341,0.645022,0.686508,1,29.0
2,149.0,"C/ PUJADES, 57B",2022.0,6.0,8.0,20.0,2022-06-08 20:59:39,2022-06-08 19:59:39,2022-06-08 18:59:39,2022-06-08 17:59:39,...,1.0,1.0,1.0,0.735043,0.491582,0.675214,0.864198,0.801347,1,30.0
3,342.0,"C/ ROC BORONAT, 134",2020.0,2.0,4.0,4.0,2020-02-04 04:55:16,2020-02-04 03:55:16,2020-02-04 02:55:16,2020-02-04 01:55:16,...,1.0,1.0,1.0,0.777778,0.898990,0.831909,0.777778,0.777778,1,30.0
4,358.0,"C/ GOMBAU, 24",2021.0,5.0,28.0,8.0,2021-05-28 08:59:30,2021-05-28 07:59:30,2021-05-28 06:59:30,2021-05-28 05:59:30,...,1.0,1.0,1.0,0.943333,0.480000,0.480000,0.513333,0.766667,1,30.0


In [22]:
testing = data.head()

In [44]:
def time_marker_creation(dataframe:dd, columns:list, weather_df:pd.DataFrame) -> dd:

    column_list = ['last_updated_formated', 'last_update_formated_H-1', 'last_update_formated_H-2', 'last_update_formated_H-3', 'last_update_formated_H-4']

    assert all(item in list(data.columns) for item in ['name', 'year'])
    assert all(column in list(data.columns) for column in column_list)

    for column in columns:
        dataframe[column + '_weathermerge'] = dataframe['year']*1000000+dataframe['month']*10000+dataframe['day']*100+dataframe['hour']
        dataframe[column + '_weathermerge'] = dataframe[column + '_weathermerge'].astype(int)

    weather_data['merge_time'] = pd.to_datetime(weather_data['timestamp'], format='%Y%m%d%H')
    weather_data['datetime'] = weather_data['timestamp'].apply(lambda x: dt.datetime.fromtimestamp(x))

    weather_data['merge_time'] = weather_data['datetime'].apply(lambda x: x.year*1000000+x.month*10000+x.day*100+x.hour)

    weather_data = weather_data.groupby(weather_data.index//2).mean()

    weather_data['mm_precip'] = weather_data['mm_precip']**2

    weather_data['merge_time'] = weather_data['merge_time'].astype(int)
    
    weather_data = weather_data.drop(columns = 'datetime')
    for column in columns:
        dataframe = pd.merge(left = dataframe, right = weather_data, left_on = column + '_weathermerge', right_on = 'merge_time',   how = 'left')
        dataframe = dataframe.drop(columns = column + '_weathermerge')
    return dataframe



In [45]:
new_dataframe = time_marker_creation(dataframe=testing, columns=['last_updated_formated', 'last_update_formated_H-1', 'last_update_formated_H-2', 'last_update_formated_H-3', 'last_update_formated_H-4'])

In [46]:
new_dataframe

,station_id,name,year,month,day,hour,last_updated_formated,last_update_formated_H-1,last_update_formated_H-2,last_update_formated_H-3,...,% Docks Available H-3,% Docks Available H-2,% Docks Available H-1,status,ttl,last_updated_formated_weathermerge,last_update_formated_H-1_weathermerge,last_update_formated_H-2_weathermerge,last_update_formated_H-3_weathermerge,last_update_formated_H-4_weathermerge
0,290.0,"PL. DELS JARDINS D'ALFÀBIA, 1",2019.0,7.0,22.0,8.0,2019-07-22 08:59:55,2019-07-22 07:59:55,2019-07-22 06:59:55,2019-07-22 05:59:55,...,0.352941,0.352941,0.504902,1,30.0,2.019072e+09,2.019072e+09,2.019072e+09,2.019072e+09,2.019072e+09
1,271.0,"VIA BARCINO, 69",2022.0,6.0,10.0,21.0,2022-06-10 21:59:45,2022-06-10 20:59:45,2022-06-10 19:59:45,2022-06-10 18:59:45,...,0.659341,0.645022,0.686508,1,29.0,2.022061e+09,2.022061e+09,2.022061e+09,2.022061e+09,2.022061e+09
2,149.0,"C/ PUJADES, 57B",2022.0,6.0,8.0,20.0,2022-06-08 20:59:39,2022-06-08 19:59:39,2022-06-08 18:59:39,2022-06-08 17:59:39,...,0.675214,0.864198,0.801347,1,30.0,2.022061e+09,2.022061e+09,2.022061e+09,2.022061e+09,2.022061e+09
3,342.0,"C/ ROC BORONAT, 134",2020.0,2.0,4.0,4.0,2020-02-04 04:55:16,2020-02-04 03:55:16,2020-02-04 02:55:16,2020-02-04 01:55:16,...,0.831909,0.777778,0.777778,1,30.0,2.020020e+09,2.020020e+09,2.020020e+09,2.020020e+09,2.020020e+09
4,358.0,"C/ GOMBAU, 24",2021.0,5.0,28.0,8.0,2021-05-28 08:59:30,2021-05-28 07:59:30,2021-05-28 06:59:30,2021-05-28 05:59:30,...,0.480000,0.513333,0.766667,1,30.0,2.021053e+09,2.021053e+09,2.021053e+09,2.021053e+09,2.021053e+09


In [69]:
weather_data = pd.read_csv('weather_data/weather.csv')

weather_data['datetime'] = weather_data['timestamp'].apply(lambda x: dt.datetime.fromtimestamp(x))

weather_data['merge_time'] = weather_data['datetime'].apply(lambda x: x.year*1000000+x.month*10000+x.day*100+x.hour)

weather_data = weather_data.groupby(weather_data.index//2).mean()

weather_data['mm_precip'] = weather_data['mm_precip']**2

example = weather_data.head()

another = pd.DataFrame({'lll':[9.00, 9.10, 8.45, 7.55, 7.50], 'aaa':['a', 'b', 'c', 'd', 'e']})

pd.merge(left = example, right = another, left_on = 'temperature', right_on = 'lll',   how = 'left')


C:\Users\ptbad\AppData\Local\Temp\ipykernel_22840\3011292691.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weather_data = weather_data.groupby(weather_data.index//2).mean()


,timestamp,mm_precip,temperature,merge_time,lll,aaa
0,1.546298e+09,0.0,9.00,2.019010e+09,9.00,a
1,1.546302e+09,0.0,9.10,2.019010e+09,9.10,b
2,1.546305e+09,0.0,8.45,2.019010e+09,8.45,c
3,1.546309e+09,0.0,7.55,2.019010e+09,NaN,NaN
4,1.546312e+09,0.0,7.50,2.019010e+09,7.50,e


In [63]:
weather_data = weather_data.groupby(weather_data.index//2).mean().head()
weather_data['merge_time'].astype(int)

C:\Users\ptbad\AppData\Local\Temp\ipykernel_22840\1873820028.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weather_data = weather_data.groupby(weather_data.index//2).mean().head()


0    2019010100
1    2019010101
2    2019010102
3    2019010103
4    2019010104
Name: merge_time, dtype: int32

In [25]:
len

15209395

In [31]:
data.station_id.unique().compute()

0      290.0
1      271.0
2      149.0
3      342.0
4      358.0
       ...  
501    199.0
502    366.0
503    457.0
504    419.0
505    431.0
Name: station_id, Length: 506, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


